## Preparing Data:

In [ ]:
import pandas as pd
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split


# Cargar el archivo de características
features_df = pd.read_csv("/home/andres_marin/Notebooks/Tesis/New features/all channels - all bands/New_alldata_allch_allbands.csv", header=None)
features = features_df.values

# Shape of the data:
print("Dimensions of the data:", features.shape)


df = pd.read_csv("/home/andres_marin/Notebooks/Tesis/labels/Labels_All_Data.csv",
                     header=None)
labels = df.iloc[:, 0].values

# Verificar la forma y la distribución de las etiquetas
print("Labels shape:", labels.shape)
print("Firts values:", labels[:10])  # Muestra las primeras 10 etiquetas

# Verificar la distribución de las etiquetas
label_distribution = Counter(labels)
print("Labels dsitribution:", label_distribution)

In [ ]:
# Shirt and sweater data:
dftrainSS = pd.read_csv("/home/andres_marin/Notebooks/Tesis/labels/Labels_Shirt_Sweater.csv",
                     header=None)
labelsSS = dftrainSS.iloc[:, 0].values

# Cargar el archivo de características (suponiendo que las características están en un formato adecuado)
featurestrainSS = pd.read_csv("/home/andres_marin/Notebooks/Tesis/Finals_tests/Final test/All channels - All bands/FinalTest_shirtsweat_allf_allch_bands.csv", header=None)
featuresSS = featurestrainSS.values

# Dividir los datos en conjuntos de entrenamiento y validación
x_div1, x_div2, y_div1, y_div2 = train_test_split(
    featuresSS, labelsSS, test_size=0.50, stratify=labelsSS, random_state=42)

features = np.vstack((features, x_div1))

# Imprimiendo las dimensiones para verificar
print("Dimensiones de la matriz de características combinada:", x_train.shape)

labels = np.concatenate((labels, y_div1))

# Verificación de las dimensiones
print("Dimensiones de los labels combinados:", y_train.shape)

- Data Division:

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y validación
x_train, x_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.3, stratify=labels, random_state=42
)

# Verificar la distribución de las clases en los conjuntos de entrenamiento y validación
train_counts = Counter(y_train)
val_counts = Counter(y_test)

print("\nDistribución de clases en el conjunto de entrenamiento:", train_counts)
print("Distribución de clases en el conjunto de validación:", val_counts)

In [ ]:
# Shirt and sweater data:
dftrainSS = pd.read_csv("/home/andres_marin/Notebooks/Tesis/labels/Labels_Shirt_Sweater.csv",
                     header=None)
labelsSS = dftrainSS.iloc[:, 0].values

# Cargar el archivo de características (suponiendo que las características están en un formato adecuado)
featurestrainSS = pd.read_csv("/home/andres_marin/Notebooks/Tesis/Finals_tests/Final test/All channels - All bands/FinalTest_shirtsweat_allf_allch_bands.csv", header=None)
featuresSS = featurestrainSS.values

# Dividir los datos en conjuntos de entrenamiento y validación
x_div1, x_div2, y_div1, y_div2 = train_test_split(
    featuresSS, labelsSS, test_size=0.50, stratify=labelsSS, random_state=42)

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import RobustScaler

# Normalizar los datos usando RobustScaler
selector = StandardScaler()
x_train2 = selector.fit_transform(x_train2)
x_train = selector.transform(x_train)
x_test = selector.transform(x_test)

# Verificar la normalización con RobustScaler
print(f'Robust Train Min: {x_train.min()}')
print(f'Robust Train Max: {x_train.max()}')
print(f'Robust Test Min: {x_test.min()}')
print(f'Robust Test Max: {x_test.max()}')

**To save the normalization scaler:**

In [ ]:
from joblib import dump, load

# Guardar el RobustScaler
dump(selector, 'scaler_allbands_allchannels.joblib')

# Classifiers:

## KNN: 
Prepare the model to study the Automatic Feature Selectors:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import optuna

def objective(trial):

    # Sugerir el número de vecinos (sólo números impares)
    n_neighbors = trial.suggest_int('n_neighbors', 1, 31, step=2)

    # Sugerir la métrica de distancia
    metric = trial.suggest_categorical('metric',['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'hamming', 'canberra', 'seuclidean'])

    # Configurar parámetros adicionales si se selecciona Minkowski o SEuclidean
    if metric == 'minkowski':
        p = trial.suggest_float('p', 1, 4)
        metric_params = {'p': p}
    elif metric == 'seuclidean':
        # Calcular el vector de varianzas de los datos de entrenamiento para seuclidean
        V = np.var(x_train, axis=0)
        metric_params = {'V': V}
    else:
        metric_params = None

    # Crear el clasificador k-NN con los parámetros sugeridos
    classifier = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, metric_params=metric_params)

    # Entrenar el clasificador en el conjunto de entrenamiento
    classifier.fit(x_train, y_train)

    # Predecir el conjunto de prueba
    y_pred = classifier.predict(x_test)

    # Evaluar el clasificador usando la precisión en el conjunto de prueba
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Crear un estudio de Optuna y encontrar los mejores parámetros
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

# Imprimir los mejores parámetros
print("Mejores parámetros: ", study.best_params)
print("Mejor precisión obtenida: ", study.best_value)

## SVM: 
Prepare the model to study the Automatic Feature Selectors: **(RFE)**

In [ ]:
import optuna
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

def objective(trial):

    C = trial.suggest_float('C', 0.01, 4, log=True)
    
    # Crear el objeto del clasificador SVC con los hiperparámetros sugeridos
    classifier_obj = SVC(kernel='linear', C=C, random_state=42)

    # Entrenar el modelo con el conjunto de entrenamiento cargado
    classifier_obj.fit(x_train, y_train)

    # Evaluar el modelo con el conjunto de prueba
    y_pred = classifier_obj.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Crear y optimizar el estudio
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)

print('Mejores parámetros:', study.best_params)
print('Mejor resultado de precisión:', study.best_value)

## LDA: 
Prepare the model to study the Automatic Feature Selectors:

In [ ]:
import optuna
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Supongamos que x_train, y_train, x_test, y_test están definidos

def objective(trial):
    # Sugerencias para los hiperparámetros
    solver = trial.suggest_categorical('solver', ['svd', 'lsqr', 'eigen'])
    shrinkage = None
    if solver in ['lsqr', 'eigen']:
        shrinkage = trial.suggest_categorical('shrinkage', ['auto', None])
        
    # Los priors pueden ser None o un arreglo que sume 1 y represente la probabilidad a priori de cada clase
    num_classes = np.unique(y_train).size
    if trial.suggest_categorical('optimize_priors', [True, False]):
        priors = np.array([trial.suggest_float(f'prior_{i}', 0.01, 1) for i in range(num_classes)])
        priors = priors / np.sum(priors)  # Normalizar para que sumen 1
    else:
        priors = None

    # Crear el objeto del clasificador LDA con los hiperparámetros sugeridos
    classifier_obj = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage, priors=priors)

    # Entrenar el modelo con el conjunto de entrenamiento
    classifier_obj.fit(x_train, y_train)

    # Evaluar el modelo con el conjunto de prueba
    y_pred = classifier_obj.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Crear un estudio de Optuna y optimizar
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Mejores parámetros:', study.best_params)
print('Mejor resultado de precisión:', study.best_value)

## Random Forest:

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    max_depth = trial.suggest_int('max_depth', 1, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])

    # Crear el objeto del clasificador RandomForest con los hiperparámetros sugeridos
    classifier_obj = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

    # Entrenar el modelo con el conjunto de entrenamiento
    classifier_obj.fit(x_train, y_train)

    # Evaluar el modelo con el conjunto de prueba
    y_pred = classifier_obj.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Crear y optimizar el estudio
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Mejores parámetros:', study.best_params)
print('Mejor resultado de precisión:', study.best_value)


# 1. Feature elmination:
1. Recursive Feature Elimination:

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import numpy as np

# Parámetros obtenidos de Optuna
solver = 'eigen'
shrinkage = 'auto'
priors = [0.5035153587365824, 0.32907437446065674]

# Normalizar priors para que sumen 1
priors = np.array(priors)
priors = priors / np.sum(priors)

# KNN:
V = np.var(x_train, axis=0)
metric_params = {'V': V}

# Models:
lda = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage, priors=priors)
svm = SVC(kernel='linear', C= 0.10863871483648994, random_state=42)
rf = RandomForestClassifier(n_estimators=779, max_depth=39, min_samples_split=9, min_samples_leaf=8, max_features='sqrt', random_state=42)
knn = KNeighborsClassifier(n_neighbors=13, metric= 'chebyshev')

**RFE for the SVM:**

In [ ]:
selectorSVM = RFECV(estimator=svm, step=1, cv=5, scoring='accuracy')
selectorSVM.fit(x_train, y_train)

# Evaluación del modelo con las características seleccionadas
print("Número óptimo de características: %d" % selectorSVM.n_features_)
print("Características seleccionadas: %s" % selectorSVM.support_)

X_train_selectedSVM = selectorSVM.transform(x_train)
X_test_selectedSVM = selectorSVM.transform(x_test)
svm.fit(X_train_selectedSVM, y_train)
y_pred = svm.predict(X_test_selectedSVM)
print("Precisión después de la selección de características: ", accuracy_score(y_test, y_pred))

**RFE for the LDA:**

In [ ]:
# Aplicar RFECV para seleccionar características para el LDA:
selectorLDA = RFECV(estimator=lda, step=1, cv=5, scoring='accuracy')
selectorLDA.fit(x_train, y_train)

# Evaluación del modelo con las características seleccionadas
print("Número óptimo de características: %d" % selectorLDA.n_features_)
print("Características seleccionadas: %s" % selectorLDA.support_)

X_train_selectedLDA = selectorLDA.transform(x_train)
X_test_selectedLDA = selectorLDA.transform(x_test)
lda.fit(X_train_selectedLDA, y_train)
y_pred = lda.predict(X_test_selectedLDA)
print("Precisión después de la selección de características: ", accuracy_score(y_test, y_pred))

**RFE for the Random Forest:**

In [ ]:
# Aplicar RFECV para seleccionar características para el LDA:
selectorRF = RFECV(estimator=rf, step=1, cv=5, scoring='accuracy')
selectorRF.fit(x_train, y_train)

# Evaluación del modelo con las características seleccionadas
print("Número óptimo de características: %d" % selectorRF.n_features_)
print("Características seleccionadas: %s" % selectorRF.support_)

X_train_selectedRF = selectorRF.transform(x_train)
X_test_selectedRF = selectorRF.transform(x_test)
rf.fit(X_train_selectedRF, y_train)
y_pred = rf.predict(X_test_selectedRF)
print("Precisión después de la selección de características: ", accuracy_score(y_test, y_pred))

*Save Selectors:*

In [ ]:
from joblib import dump, load
# Guardar el selector en un archivo
dump(selectorSVM, 'selector_rfecv_svm.joblib')
dump(selectorLDA, 'selector_rfecv_lda.joblib')
dump(selectorRF, 'selector_rfecv_rf.joblib')

# 2. From the Random Forest:

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import numpy as np

# Parámetros obtenidos de Optuna
solver = 'eigen'
shrinkage = 'auto'
priors = [0.5035153587365824, 0.32907437446065674]

# Normalizar priors para que sumen 1
priors = np.array(priors)
priors = priors / np.sum(priors)

# KNN:
V = np.var(x_train, axis=0)
metric_params = {'V': V}

# Models:
lda = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage, priors=priors)
svm = SVC(kernel='linear', C= 0.10863871483648994, random_state=42)
rf = RandomForestClassifier(n_estimators=779, max_depth=39, min_samples_split=9, min_samples_leaf=8, max_features='sqrt', random_state=42)
knn = KNeighborsClassifier(n_neighbors=13, metric= 'chebyshev')

In [ ]:
rf.fit(x_train, y_train)

*Using the information manually:*

In [ ]:
sfm_mean = SelectFromModel(rf, threshold='mean')
X_train_mean = sfm_mean.fit_transform(x_train, y_train)

# Umbral como la mediana de las importancias
sfm_median = SelectFromModel(rf, threshold='median')
X_train_median = sfm_median.fit_transform(x_train, y_train)

# Umbral personalizado, por ejemplo, 1.5 veces la media de las importancias
mean_importance = rf.feature_importances_.mean()
sfm_custom = SelectFromModel(rf, threshold=1.5 * mean_importance)
X_train_custom = sfm_custom.fit_transform(x_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

# Definir los umbrales a evaluar
thresholds = ['mean', 'median', 0.01]
results = []

# Configurar la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Ciclo para evaluar cada umbral
for threshold in thresholds:
    sfm = SelectFromModel(rf, threshold=threshold)
    X_train_transformed = sfm.fit_transform(x_train, y_train)
    scores = cross_val_score(rf, X_train_transformed, y_train, cv=cv, scoring='accuracy')
    results.append({
        'threshold': threshold,
        'mean_accuracy': np.mean(scores),
        'std_dev': np.std(scores),
        'num_features': X_train_transformed.shape[1]
    })

# Mostrar resultados
for result in results:
    print(f"Umbral: {result['threshold']}, Precisión media: {result['mean_accuracy']:.4f}, Desv. Est.: {result['std_dev']:.4f}, Núm. Características: {result['num_features']}")


**Saving the Selector Model:**

In [ ]:
# Get the model
sfm = SelectFromModel(rf, threshold='median')
sfm.fit(x_train, y_train)

# Save the model:
from joblib import dump
dump(sfm, 'sfmMean.joblib')

**Testing the models with the selector:**

In [ ]:
from joblib import load
sfm=load('/home/andres_marin/Notebooks/Tesis/Finals_tests/Selectors/2. Random Forest/sfm.joblib')

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import numpy as np

# Parámetros obtenidos de Optuna
solver = 'eigen'
shrinkage = 'auto'
priors = [0.9954204041462262, 0.5897282542157889]

# Normalizar priors para que sumen 1
priors = np.array(priors)
priors = priors / np.sum(priors)

# Models:
lda = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage, priors=priors)
svm = SVC(kernel='linear', C=0.03145231545300702, random_state=42)
rf = RandomForestClassifier(n_estimators=792, max_depth=2, min_samples_split=5, min_samples_leaf=8, max_features='sqrt', random_state=42)

In [ ]:
# Getting the new features:
x_trainRandomForest = sfm.transform(x_train)
x_testRandomForest = sfm.transform(x_test)

In [ ]:
# Training the models:
svm.fit(x_trainRandomForest, y_train)
lda.fit(x_trainRandomForest, y_train)
rf.fit(x_trainRandomForest, y_train)

In [ ]:
# Testing the models:

# SVM:
y_pred = svm.predict(x_testRandomForest)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with SVM: {accuracy:.4f}")

# LDA:
y_pred = lda.predict(x_testRandomForest)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with LDA: {accuracy:.4f}")

# Random Forest:
y_pred = rf.predict(x_testRandomForest)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with Random Forest: {accuracy:.4f}")

# 3. Sequential Feature Selector

**Load the models:**

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import numpy as np

# Parámetros obtenidos de Optuna
solver = 'eigen'
shrinkage = 'auto'
priors = [0.9954204041462262, 0.5897282542157889]

# Normalizar priors para que sumen 1
priors = np.array(priors)
priors = priors / np.sum(priors)

# KNN:
V = np.var(x_train, axis=0)
metric_params = {'V': V}

# Models:
lda = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage, priors=priors)
svm = SVC(kernel='linear', C=0.03145231545300702, random_state=42)
rf = RandomForestClassifier(n_estimators=792, max_depth=2, min_samples_split=5, min_samples_leaf=8, max_features='sqrt', random_state=42)
knn = KNeighborsClassifier(n_neighbors=15, metric= 'seuclidean', metric_params=metric_params)

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Configurar SFS
sfsLDA = SFS(lda,
          k_features='best',  # Selecciona el número de características que resulte en el mejor rendimiento
          forward=True,  # True para selección hacia adelante, False para selección hacia atrás
          floating=False,  # 'Floating' puede eliminar o añadir en cada paso dependiendo del criterio
          scoring='accuracy',  # Métrica de rendimiento
          cv=5)  # Validación cruzada
sfsLDA.fit(x_train, y_train)

# Resultados de la selección de características
print('Número de características seleccionadas:', len(sfsLDA.k_feature_names_))
print('Mejores características:', sfsLDA.k_feature_names_)

In [ ]:
from joblib import dump
dump(sfsLDA, 'sfsLDA.joblib')

In [ ]:
# Configurar SFS
sfsSVM = SFS(svm,
          k_features='best',  # Selecciona el número de características que resulte en el mejor rendimiento
          forward=True,  # True para selección hacia adelante, False para selección hacia atrás
          floating=False,  # 'Floating' puede eliminar o añadir en cada paso dependiendo del criterio
          scoring='accuracy',  # Métrica de rendimiento
          cv=5)  # Validación cruzada
sfsSVM.fit(x_train, y_train)

# Resultados de la selección de características
print('Número de características seleccionadas:', len(sfsSVM.k_feature_names_))
print('Mejores características:', sfsSVM.k_feature_names_)

In [ ]:
from joblib import dump
dump(sfsSVM, 'sfsLDA.joblib')

In [ ]:
# Configurar SFS
sfsKNN = SFS(knn,
          k_features='best',  # Selecciona el número de características que resulte en el mejor rendimiento
          forward=True,  # True para selección hacia adelante, False para selección hacia atrás
          floating=False,  # 'Floating' puede eliminar o añadir en cada paso dependiendo del criterio
          scoring='accuracy',  # Métrica de rendimiento
          cv=5)  # Validación cruzada
sfsKNN.fit(x_train, y_train)

# Resultados de la selección de características
print('Número de características seleccionadas:', len(sfsKNN.k_feature_names_))
print('Mejores características:', sfsKNN.k_feature_names_)

In [ ]:
from joblib import dump
dump(sfsKNN, 'sfsLDA.joblib')

In [ ]:
# Configurar SFS
sfsRF = SFS(rf,
          k_features='best',  # Selecciona el número de características que resulte en el mejor rendimiento
          forward=True,  # True para selección hacia adelante, False para selección hacia atrás
          floating=False,  # 'Floating' puede eliminar o añadir en cada paso dependiendo del criterio
          scoring='accuracy',  # Métrica de rendimiento
          cv=5)  # Validación cruzada
sfsRF.fit(x_train, y_train)

# Resultados de la selección de características
print('Número de características seleccionadas:', len(sfsRF.k_feature_names_))
print('Mejores características:', sfsRF.k_feature_names_)

In [ ]:
from joblib import dump
dump(sfsRF, 'sfsLDA.joblib')

# 4. L1-based feature selection (Lasso)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

lasso = Lasso(alpha=0.0007)  # Ajusta el valor de alpha según necesidad
lasso.fit(x_train, y_train)

model = SelectFromModel(lasso, prefit=True)
X_train_selected = model.transform(x_train)
X_test_selected = model.transform(x_test)

# Obtener la máscara de las características seleccionadas
selected_features = model.get_support()
# Contar las características seleccionadas
number_of_selected_features = selected_features.sum()

print("Número de características seleccionadas:", number_of_selected_features)

In [ ]:
# Save the model:
from joblib import dump
dump(model, 'L1_Selector.joblib')

# 5. Mutal Information:

In [ ]:
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
import numpy as np

# Parámetros obtenidos de Optuna
solver = 'eigen'
shrinkage = 'auto'
priors = [0.5035153587365824, 0.32907437446065674]

# Normalizar priors para que sumen 1
priors = np.array(priors)
priors = priors / np.sum(priors)

# KNN:
V = np.var(x_train, axis=0)
metric_params = {'V': V}

# Models:
lda = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage, priors=priors)
svm = SVC(kernel='linear', C= 0.10863871483648994, random_state=42)
rf = RandomForestClassifier(n_estimators=779, max_depth=39, min_samples_split=9, min_samples_leaf=8, max_features='sqrt', random_state=42)
knn = KNeighborsClassifier(n_neighbors=13, metric= 'chebyshev')

In [ ]:
# Lista para guardar los puntajes promedio de validación cruzada
mean_scores = []

# Rango de 'k' a probar
k_values = range(1, 280)  # Puedes ajustar este rango según la cantidad de características iniciales

# Inicializar la variable para el selector óptimo
best_selector = None

for k in k_values:
    # Selección de las 'k' mejores características basada en información mutua
    selector = SelectKBest(mutual_info_classif, k=k)
    X_train_selected = selector.fit_transform(x_train, y_train)
    
    # Crear y evaluar el clasificador usando validación cruzada
    clf = rf
    scores = cross_val_score(clf, X_train_selected, y_train, cv=5)
    mean_scores.append(np.mean(scores))
    
    # Guardar el selector óptimo
    if np.mean(scores) == max(mean_scores):
        best_selector = selector

# Encontrar el valor de 'k' que maximizó el puntaje de validación cruzada
optimal_k = k_values[np.argmax(mean_scores)]
print("Óptimo número de características:", optimal_k)

In [ ]:
# Save the model:
from joblib import dump
dump(best_selector, 'MS_rf.joblib')

# 6. PCA:

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

# Inicializar PCA para que retenga el 95% de la varianza
pca = PCA(n_components=0.99)

# Ajustar PCA al dataset escalado
X_pca = pca.fit_transform(x_train)

# Mostrar el nuevo número de componentes
print("Número de componentes seleccionados:", pca.n_components_)
print("Varianza explicada por cada componente:", pca.explained_variance_ratio_)
print("Varianza total explicada:", sum(pca.explained_variance_ratio_))


In [ ]:
# Save the model:
from joblib import dump
dump(pca, 'pca.joblib')